In [1]:
epochs = 10

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

In [3]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2070 SUPER, compute capability 7.5
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale
Compute dtype: float16
Variable dtype: float32


In [4]:
train_dir = "D:/Ocular toxoplasmosis binzarized/Oculur Toxoplasmosis/train"
test_dir = "D:/Ocular toxoplasmosis binzarized/Oculur Toxoplasmosis/val"

In [5]:
input_shape = (112, 112)

datagen_train = ImageDataGenerator(rescale=1./255,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=True,
                                  vertical_flip=False)


datagen_test = ImageDataGenerator(rescale=1./255)


batch_size = 16
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)

generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)
steps_per_epoch = generator_train.n / batch_size
steps_test = generator_test.n / batch_size

Found 366 images belonging to 2 classes.
Found 83 images belonging to 2 classes.


In [6]:
def model_creator(model):
    transfer_layer = model.get_layer(model.get_layer(index = -1).name)
    conv_model = Model(inputs=model.input, outputs=transfer_layer.output)
    new_model = Sequential()
    new_model.add(conv_model)
    new_model.add(Flatten())
    new_model.add(Dense(2, activation='softmax'))
    optimizer = Adam(lr=1e-5)
    loss = 'categorical_crossentropy'
    metrics = ['categorical_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]
    new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return new_model

In [7]:
model = tf.keras.applications.VGG19(include_top=False,input_shape=(112, 112, 3), weights='imagenet')
vgg19 = model_creator(model)
history_vgg19 = vgg19.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



D:\Anaconda\envs\tf_gpu_final\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
C:\Users\User\AppData\Local\Temp\ipykernel_9428\183852360.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_vgg19 = vgg19.fit_generator(generator=generator_train,


Epoch 1/10
22/22 [==============================] - 26s 849ms/step - loss: 0.5365 - categorical_accuracy: 0.7158 - precision: 0.7158 - recall: 0.7158 - auc: 0.8043 - val_loss: 0.5607 - val_categorical_accuracy: 0.7108 - val_precision: 0.7108 - val_recall: 0.7108 - val_auc: 0.7836
Epoch 2/10
22/22 [==============================] - 17s 738ms/step - loss: 0.3073 - categorical_accuracy: 0.8989 - precision: 0.8989 - recall: 0.8989 - auc: 0.9552 - val_loss: 0.4739 - val_categorical_accuracy: 0.8072 - val_precision: 0.8072 - val_recall: 0.8072 - val_auc: 0.8588
Epoch 3/10
22/22 [==============================] - 16s 716ms/step - loss: 0.2120 - categorical_accuracy: 0.9208 - precision: 0.9208 - recall: 0.9208 - auc: 0.9743 - val_loss: 0.6588 - val_categorical_accuracy: 0.7229 - val_precision: 0.7229 - val_recall: 0.7229 - val_auc: 0.8138
Epoch 4/10
22/22 [==============================] - 17s 732ms/step - loss: 0.1725 - categorical_accuracy: 0.9317 - precision: 0.9317 - recall: 0.9317 - auc: 

In [8]:
model = tf.keras.applications.InceptionV3(include_top=False,input_shape=(112, 112, 3), weights='imagenet')
inception = model_creator(model)
history_inception = inception.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



C:\Users\User\AppData\Local\Temp\ipykernel_9428\576356427.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_inception = inception.fit_generator(generator=generator_train,


Epoch 1/10
22/22 [==============================] - 29s 806ms/step - loss: 0.7903 - categorical_accuracy: 0.5301 - precision_1: 0.5301 - recall_1: 0.5301 - auc_1: 0.5320 - val_loss: 1.0482 - val_categorical_accuracy: 0.6024 - val_precision_1: 0.6024 - val_recall_1: 0.6024 - val_auc_1: 0.5795
Epoch 2/10
22/22 [==============================] - 16s 718ms/step - loss: 0.6328 - categorical_accuracy: 0.6366 - precision_1: 0.6366 - recall_1: 0.6366 - auc_1: 0.7040 - val_loss: 0.9488 - val_categorical_accuracy: 0.6024 - val_precision_1: 0.6024 - val_recall_1: 0.6024 - val_auc_1: 0.6193
Epoch 3/10
22/22 [==============================] - 17s 728ms/step - loss: 0.5594 - categorical_accuracy: 0.7268 - precision_1: 0.7268 - recall_1: 0.7268 - auc_1: 0.7832 - val_loss: 0.9136 - val_categorical_accuracy: 0.5663 - val_precision_1: 0.5663 - val_recall_1: 0.5663 - val_auc_1: 0.5712
Epoch 4/10
22/22 [==============================] - 17s 742ms/step - loss: 0.5634 - categorical_accuracy: 0.7404 - precis

In [9]:
model = tf.keras.applications.MobileNet(include_top=False,input_shape=(112, 112, 3), weights='imagenet')
mobile = model_creator(model)
history_mobile = mobile.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



C:\Users\User\AppData\Local\Temp\ipykernel_9428\789522733.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_mobile = mobile.fit_generator(generator=generator_train,


Epoch 1/10
22/22 [==============================] - 21s 750ms/step - loss: 1.3098 - categorical_accuracy: 0.5273 - precision_2: 0.5273 - recall_2: 0.5273 - auc_2: 0.5483 - val_loss: 1.2947 - val_categorical_accuracy: 0.4217 - val_precision_2: 0.4217 - val_recall_2: 0.4217 - val_auc_2: 0.4307
Epoch 2/10
22/22 [==============================] - 17s 754ms/step - loss: 1.0583 - categorical_accuracy: 0.6311 - precision_2: 0.6311 - recall_2: 0.6311 - auc_2: 0.6505 - val_loss: 0.9955 - val_categorical_accuracy: 0.5542 - val_precision_2: 0.5542 - val_recall_2: 0.5542 - val_auc_2: 0.5814
Epoch 3/10
22/22 [==============================] - 17s 734ms/step - loss: 0.7253 - categorical_accuracy: 0.7213 - precision_2: 0.7213 - recall_2: 0.7213 - auc_2: 0.7856 - val_loss: 0.9245 - val_categorical_accuracy: 0.5904 - val_precision_2: 0.5904 - val_recall_2: 0.5904 - val_auc_2: 0.6190
Epoch 4/10
22/22 [==============================] - 17s 737ms/step - loss: 0.6813 - categorical_accuracy: 0.7295 - precis

In [10]:
model = tf.keras.applications.DenseNet121(include_top=False,input_shape=(112, 112, 3), weights='imagenet')
dense = model_creator(model)
history_dense = dense.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



C:\Users\User\AppData\Local\Temp\ipykernel_9428\3413688818.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_dense = dense.fit_generator(generator=generator_train,


Epoch 1/10
22/22 [==============================] - 37s 898ms/step - loss: 0.8953 - categorical_accuracy: 0.6175 - precision_3: 0.6175 - recall_3: 0.6175 - auc_3: 0.6392 - val_loss: 1.5069 - val_categorical_accuracy: 0.5663 - val_precision_3: 0.5663 - val_recall_3: 0.5663 - val_auc_3: 0.5750
Epoch 2/10
22/22 [==============================] - 17s 756ms/step - loss: 0.5201 - categorical_accuracy: 0.7213 - precision_3: 0.7213 - recall_3: 0.7213 - auc_3: 0.8313 - val_loss: 1.1605 - val_categorical_accuracy: 0.5422 - val_precision_3: 0.5422 - val_recall_3: 0.5422 - val_auc_3: 0.6115
Epoch 3/10
22/22 [==============================] - 17s 754ms/step - loss: 0.4625 - categorical_accuracy: 0.7869 - precision_3: 0.7869 - recall_3: 0.7869 - auc_3: 0.8692 - val_loss: 0.9434 - val_categorical_accuracy: 0.5783 - val_precision_3: 0.5783 - val_recall_3: 0.5783 - val_auc_3: 0.6317
Epoch 4/10
22/22 [==============================] - 17s 746ms/step - loss: 0.3908 - categorical_accuracy: 0.8060 - precis

In [11]:
model = tf.keras.applications.Xception(include_top=False,input_shape=(112, 112, 3), weights='imagenet')
xcep = model_creator(model)
history_xcep = xcep.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



C:\Users\User\AppData\Local\Temp\ipykernel_9428\28214634.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_xcep = xcep.fit_generator(generator=generator_train,


Epoch 1/10
22/22 [==============================] - 26s 803ms/step - loss: 0.6917 - categorical_accuracy: 0.5574 - precision_4: 0.5549 - recall_4: 0.5519 - auc_4: 0.5852 - val_loss: 0.6209 - val_categorical_accuracy: 0.6386 - val_precision_4: 0.6341 - val_recall_4: 0.6265 - val_auc_4: 0.7034
Epoch 2/10
22/22 [==============================] - 18s 772ms/step - loss: 0.5975 - categorical_accuracy: 0.7049 - precision_4: 0.7068 - recall_4: 0.7049 - auc_4: 0.7477 - val_loss: 0.5865 - val_categorical_accuracy: 0.6627 - val_precision_4: 0.6627 - val_recall_4: 0.6627 - val_auc_4: 0.7503
Epoch 3/10
22/22 [==============================] - 17s 747ms/step - loss: 0.5331 - categorical_accuracy: 0.7568 - precision_4: 0.7589 - recall_4: 0.7568 - auc_4: 0.8333 - val_loss: 0.5573 - val_categorical_accuracy: 0.6988 - val_precision_4: 0.6988 - val_recall_4: 0.6988 - val_auc_4: 0.7841
Epoch 4/10
22/22 [==============================] - 17s 747ms/step - loss: 0.4709 - categorical_accuracy: 0.7842 - precis

In [12]:
model = tf.keras.applications.VGG16(include_top=False,input_shape=(112, 112, 3), weights='imagenet')
vgg16 = model_creator(model)
history_vgg16 = vgg16.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



C:\Users\User\AppData\Local\Temp\ipykernel_9428\388627917.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_vgg16 = vgg16.fit_generator(generator=generator_train,


Epoch 1/10
22/22 [==============================] - 21s 791ms/step - loss: 0.6013 - categorical_accuracy: 0.6858 - precision_5: 0.6858 - recall_5: 0.6858 - auc_5: 0.7399 - val_loss: 0.5420 - val_categorical_accuracy: 0.7590 - val_precision_5: 0.7590 - val_recall_5: 0.7590 - val_auc_5: 0.8358
Epoch 2/10
22/22 [==============================] - 17s 741ms/step - loss: 0.4014 - categorical_accuracy: 0.8279 - precision_5: 0.8279 - recall_5: 0.8279 - auc_5: 0.9143 - val_loss: 0.5569 - val_categorical_accuracy: 0.6867 - val_precision_5: 0.6867 - val_recall_5: 0.6867 - val_auc_5: 0.7778
Epoch 3/10
22/22 [==============================] - 17s 739ms/step - loss: 0.2752 - categorical_accuracy: 0.8934 - precision_5: 0.8934 - recall_5: 0.8934 - auc_5: 0.9618 - val_loss: 0.3548 - val_categorical_accuracy: 0.8072 - val_precision_5: 0.8072 - val_recall_5: 0.8072 - val_auc_5: 0.9162
Epoch 4/10
22/22 [==============================] - 17s 737ms/step - loss: 0.2073 - categorical_accuracy: 0.9071 - precis